# Clustering Code Along

We'll be working with a real data set about seeds, from UCI repository: https://archive.ics.uci.edu/ml/datasets/seeds.

The examined group comprised kernels belonging to three different varieties of wheat: Kama, Rosa and Canadian, 70 elements each, randomly selected for 
the experiment. High quality visualization of the internal kernel structure was detected using a soft X-ray technique. It is non-destructive and considerably cheaper than other more sophisticated imaging techniques like scanning microscopy or laser technology. The images were recorded on 13x18 cm X-ray KODAK plates. Studies were conducted using combine harvested wheat grain originating from experimental fields, explored at the Institute of Agrophysics of the Polish Academy of Sciences in Lublin. 

The data set can be used for the tasks of classification and cluster analysis.


Attribute Information:

To construct the data, seven geometric parameters of wheat kernels were measured: 
1. area A, 
2. perimeter P, 
3. compactness C = 4*pi*A/P^2, 
4. length of kernel, 
5. width of kernel, 
6. asymmetry coefficient 
7. length of kernel groove. 
All of these parameters were real-valued continuous.

Let's see if we can cluster them in to 3 groups with K-means!

In [23]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [25]:
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.csv("seeds_dataset.csv",header=True,inferSchema=True)

In [26]:
dataset.head()

Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)

In [27]:
dataset.toPandas().head()

,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_groove
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175


In [28]:
dataset.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|summary|              area|         perimeter|         compactness|   length_of_kernel|   width_of_kernel|asymmetry_coefficient|   length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|  count|               210|               210|                 210|                210|               210|                  210|                210|
|   mean|14.847523809523816|14.559285714285718|  0.8709985714285714|  5.628533333333335| 3.258604761904762|   3.7001999999999997|  5.408071428571429|
| stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867|   1.5035589702547392|0.49148049910240543|
|    min|             10.59|             12.41|              0.8081|              4.899|            

## Format the Data

In [29]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [30]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [31]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [32]:
final_data = vec_assembler.transform(dataset)

In [33]:
final_data.toPandas().head()

,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_groove,features
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,"[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]"
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,"[14.88, 14.57, 0.8811, 5.553999999999999, 3.33..."
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,"[14.29, 14.09, 0.905, 5.291, 3.336999999999999..."
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,"[13.84, 13.94, 0.8955, 5.324, 3.37899999999999..."
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,"[16.14, 14.99, 0.9034, 5.6579999999999995, 3.5..."


## Scale the Data
It is a good idea to scale our data to deal with the curse of dimensionality: https://en.wikipedia.org/wiki/Curse_of_dimensionality

In [34]:
from pyspark.ml.feature import StandardScaler

In [35]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [36]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [37]:
# Normalize each feature to have unit standard deviation.
final_data = scalerModel.transform(final_data)

In [38]:
final_data.toPandas().head()

,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_groove,features,scaledFeatures
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,"[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]","[5.244527953320284, 11.363299389287777, 36.860..."
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,"[14.88, 14.57, 0.8811, 5.553999999999999, 3.33...","[5.113930271651758, 11.156554723849252, 37.288..."
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,"[14.29, 14.09, 0.905, 5.291, 3.336999999999999...","[4.911160186955888, 10.789008651958541, 38.299..."
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,"[13.84, 13.94, 0.8955, 5.324, 3.37899999999999...","[4.756505037611581, 10.674150504492696, 37.897..."
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,"[16.14, 14.99, 0.9034, 5.6579999999999995, 3.5...","[5.546964689815818, 11.478157536753622, 38.232..."


## Train the Model and Evaluate

In [39]:
# Trains a k-means model.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [40]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(final_data)
print("Within Set Sum of Squared Errors = " + str(wssse))
# This method is not used in Spark 3.0, see below instead

AttributeError: 'KMeansModel' object has no attribute 'computeCost'

In [43]:
# Make predictions 
predictions = model.transform(final_data)
from pyspark.ml.evaluation import ClusteringEvaluator
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.616267393520126


In [46]:
predictions.toPandas().head()

,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_of_groove,features,scaledFeatures,prediction
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,"[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]","[5.244527953320284, 11.363299389287777, 36.860...",2
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,"[14.88, 14.57, 0.8811, 5.553999999999999, 3.33...","[5.113930271651758, 11.156554723849252, 37.288...",2
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,"[14.29, 14.09, 0.905, 5.291, 3.336999999999999...","[4.911160186955888, 10.789008651958541, 38.299...",2
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,"[13.84, 13.94, 0.8955, 5.324, 3.37899999999999...","[4.756505037611581, 10.674150504492696, 37.897...",2
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,"[16.14, 14.99, 0.9034, 5.6579999999999995, 3.5...","[5.546964689815818, 11.478157536753622, 38.232...",2


In [41]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 4.06660859 10.14191893 35.84098009 11.81592066  7.52397236  3.1823335
 10.39801233]
[ 6.35645488 12.40730852 37.41990178 13.93860446  9.7892399   2.41585013
 12.29286107]
[ 4.93382436 10.94691274 37.30542404 12.41332714  8.60366812  1.82917353
 10.40106154]


In [42]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
+----------+
only showing top 20 rows



Now you are ready for your consulting Project!
# Great Job!